<a href="https://colab.research.google.com/github/HAD1M/Coursera-DS-Capstone-Project/blob/master/Hands_on_machine_learning_mnist_kazee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

Pada contoh ini kita akan membuat model machine learning **mnist** menggunakan tf.keras dan Cloud TPU yang dapat mengklasifikasi gambar-gambar tulisan tangan berupa angka.

## Inisiasi library

In [0]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import os

resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

## Load data yang digunakan untuk training model machine learning

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [0]:
print('Banyak data training : {}'.format(len(x_train)))
print('Banyak data test : {}'.format(len(x_test)))

In [0]:
LABEL_NAMES = {0 : '0', 1 : '1', 2 : '2', 3 : '3', 4 : '4', 5 : '5', 6 : '6', 7 : '7', 8 : '8', 9 : '9'}
print(LABEL_NAMES)

## Plot data training

In [0]:
##plot xtrain
plt.figure(figsize=(18,12))
for n, image in enumerate(x_train[:12]):
  plt.subplot(3,4,n+1)
  plt.imshow(image, cmap='gray')
  plt.grid(False)
  plt.axis('off')
  plt.title(LABEL_NAMES.get(y_train[n]), fontsize=20)

## Plot data test

In [0]:
##plot x test
plt.figure(figsize=(18,12))
for n, image in enumerate(x_test[:12]):
  plt.subplot(3,4,n+1)
  plt.imshow(image, cmap='gray')
  plt.grid(False)
  plt.axis('off')
  plt.title(LABEL_NAMES.get(y_test[n]))

In [0]:
# mengubah bentuk input
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

## Membuat model

In [0]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  return model

In [0]:
with strategy.scope():
  model = create_model()
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])

## Training model

In [0]:
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

# save model
model.save_weights('./mnist.h5', overwrite=True)

## Prediksi

In [0]:
# load model yang sudah ditraining
cpu_model = create_model()
cpu_model.load_weights('./mnist.h5')

In [0]:
# prediksi 12 gambar pada data test
predictions = cpu_model.predict(x_test[:12])

In [0]:
# contoh output prediksi model machine learning
print(predictions[0])

## Plot hasil prediksi

In [0]:
plt.figure(figsize=(18,12))
for n, image in enumerate(np.squeeze(x_test[:12])):
  plt.subplot(3,4,n+1)
  plt.imshow(image)
  plt.grid(False)
  plt.axis('off')
  plt.title('{}, conf_score : {:.3}'.format(LABEL_NAMES.get(np.argmax(predictions[n])), np.max(predictions[n])) )